We will compute matrix A to represent our (noiseless) imaging model.

In [1]:
import numpy as np
import pandas as pd
from data_objects import Fluorophore, get_fluorophore_list

In [2]:
illumination_wavelengths = [405, 488, 561, 637]
wavelength_range = (400,900)
bin_width = 10
fluorophore_list = get_fluorophore_list(wavelength_range=wavelength_range)
spectra_list = list(map(Fluorophore.get_spectra, fluorophore_list))

bins_list = [*range(*wavelength_range, 10)]
total_bins = len(illumination_wavelengths)*bins_list

### Function to solve emission bin area using $i$ and $j$

Corresponding lambda_min and lambda_max to the min and max wavelengths in channel i in a list of the 200 total bins

In [3]:
def calc_emission_bin(i, j):
    df_spectra = spectra_list[j]
    lambda_min = total_bins[i]
    lambda_max = lambda_min + 10
    bin_i = range(lambda_min, lambda_max)
    df_sliced_data = df_spectra.loc[df_spectra['wavelength'].isin(bin_i)]
    # print(df_sliced_data)
    sliced_data_emmision = df_sliced_data['emission']
    emission_bin = np.trapz(sliced_data_emmision)
    # print('Emission bin sum for i = ', i, 'and j = ', j, 'is', emission_bin)
    return emission_bin

### Function to calculate the excitation values from the illumination wavelengths

Excitation value corresponding to excitation wavelengths for each protein.

In [4]:
def calc_excitation(i, j): 
    if i >= len(illumination_wavelengths) * len(bins_list):
        illumination_wavelength = illumination_wavelengths[i // len(bins_list)]
    else:
        print('Invalid i value')
    
    return spectra_list[j].excitation[illumination_wavelength]

### Function to index over brightness values using a given $j$

In [5]:
def calc_brightness(j):
    brightness = fluorophore_list[j].brightness
    return brightness